In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')

In [ ]:
train

Total of 9957 rows in the training set. Let's calculate the NaN values in all the columns. Columns with high % of NaN can be dropped, wit 30-40% of data and rest of it as average or 0 doesn't make sense, better to drop those columns. Only 5 columns have NaN values, rather than worrying about it, let's drop them. Very few features going away anyways. 

Will need to drop the same 5 columns from test set also.

In [ ]:
train=train.dropna(axis=1)

In [ ]:
missing_col=set(test)-set(train)
test=test.drop(columns=missing_col,axis=1)

In [ ]:
xtrain=train.copy()
xtest=test.copy()

In [ ]:
ytrain=xtrain['Target']
ytrain=ytrain.as_matrix()

In [ ]:
xtrain=xtrain.drop(columns=['Id','Target','idhogar','dependency','edjefe','edjefa'])
xtest=xtest.drop(columns=['Id','idhogar','dependency','edjefe','edjefa'])

Let us use Light GBM (microsoft variant of XGBoost for prediction). Based on our experience, XGBoost gives better prediction results than Deep Learning, so trying Light GBM. Can also use XG Boost and then see the difference.

In [ ]:
import xgboost as xgb

model=xgb.XGBClassifier(eta=0.2)
model.fit(xtrain,ytrain)

ypred=model.predict(xtest)

In [ ]:
ytrain

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
from keras.layers import Dropout
from keras.utils import to_categorical

ytrain=to_categorical(ytrain)

model_d=models.Sequential()
model_d.add(layers.Dense(128,activation='relu',input_shape=(xtrain.shape[1],)))
model_d.add(Dropout(0.2))
model_d.add(layers.Dense(128,activation='relu'))
model_d.add(Dropout(0.2))
model_d.add(layers.Dense(5,activation='softmax'))

rmsprop=optimizers.RMSprop(lr=0.01)

model_d.compile(optimizer=rmsprop,loss='categorical_crossentropy',metrics=['accuracy'])

model_d.fit(xtrain,ytrain,epochs=32,batch_size=32)

ypred_d=model_d.predict(xtest)

In [ ]:
submission=pd.read_csv('../input/sample_submission.csv')
submission['Target']=ypred

submission.to_csv('sample_submission.csv',index=False)

In [ ]:
ypred_d
prediction_deep=[]
from numpy import argmax

for x in ypred_d:
    prediction_deep.append(np.argmax(x))

In [ ]:
submission_d=pd.read_csv('../input/sample_submission.csv')
submission_d['Target']=prediction_deep

submission_d.to_csv('sample_submission_deeplearning.csv',index=False)